In [50]:
import sys
import socket
import json
import pandas as pd
from pandas.io.json import json_normalize
import time
sys.version

'3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]'

In [95]:
def recvall(sock):
    BUFF_SIZE = 4096
    data = b''
    while True:
        part = sock.recv(BUFF_SIZE)
        data += part
        if ']' in part.decode("utf-8"):
            break   
    return data

In [175]:
#spark streamming
i = 1
while i < 2:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect(('pub-vrs.adsbexchange.com', 32005))
    data = recvall(s)
    df = json_normalize(json.loads(data.decode("utf-8").split('\"acList\":')[1].split(']')[0] + ']'))
    print(df[~df.Call.isna()][['Call','Lat','Long']].head())
    time.sleep(10)
    s.close()
    #print('Received', repr(data))
    i += 1


        Call        Lat       Long
2   00000000  44.570450  26.078278
4   00000000  44.571918  26.109125
5   00000000  44.500854  26.067225
7   00000000  44.574978  26.096985
10  00000000  44.566574  26.116261


In [1]:
import plotly.offline as pyo
import plotly.graph_objs as go
pyo.init_notebook_mode(connected=True)

data = [ dict(
        type = 'scattergeo',
        #locationmode = 'USA-states',
        lon = df[~df.Call.isna()]['Long'],
        lat = df[~df.Call.isna()]['Lat'],
        text = df[~df.Call.isna()]['Call'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            )
            #colorscale = scl,
            #cmin = 0,
            #color = df['cnt'],
            #cmax = df['cnt'].max(),
            #colorbar=dict(
            #    title="Incoming flightsFebruary 2011"
            #)
        ))]

layout = dict(
        #title = 'US airspace ADS traffic (Hover for aircraft names)',
        colorbar = True,
        geo = dict(
            scope='world',
            landcolor = "rgb(212, 212, 212)",
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
            projection=dict( type='eckert4'),
            showland = True,
            autosize = True,
            margin=go.Margin(
                        l=50,
                        r=50,
                        b=0,
                        t=0,
                        pad=0,
                        autoexpand = True),
            #countrywidth = 0.5,
            #subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
pyo.iplot( fig, validate=False, filename='d3-airports' )

NameError: name 'df' is not defined

In [22]:
import urllib.request, json 
with urllib.request.urlopen("https://opensky-network.org/api/states/all?lamin=45.8389&lomin=5.9962&lamax=47.8229&lomax=10.5226") as url:
    data = json.loads(url.read().decode())

In [58]:
from opensky_api import OpenSkyApi

api = OpenSkyApi()
states = api.get_states(bbox=(45.8389, 47.8229, 5.9962, 10.5226))
data = json.dumps([s.__dict__ for s in states.states])

In [59]:
data.encode('utf-8')

b'[{"icao24": "4b1801", "callsign": "SWR247C ", "origin_country": "Switzerland", "time_position": 1534676179, "last_contact": 1534676179, "longitude": 7.7179, "latitude": 47.1443, "geo_altitude": 5669.28, "on_ground": false, "velocity": 202.85, "heading": 239.86, "vertical_rate": 8.45, "sensors": null, "baro_altitude": 6019.8, "squawk": "3021", "spi": false, "position_source": 0}, {"icao24": "4b1884", "callsign": "SWR242  ", "origin_country": "Switzerland", "time_position": 1534676179, "last_contact": 1534676179, "longitude": 9.9958, "latitude": 47.4537, "geo_altitude": 5791.2, "on_ground": false, "velocity": 216.27, "heading": 100, "vertical_rate": 7.48, "sensors": null, "baro_altitude": 6080.76, "squawk": "3012", "spi": false, "position_source": 0}, {"icao24": "3c5ef5", "callsign": "EWG4TX  ", "origin_country": "Germany", "time_position": 1534676179, "last_contact": 1534676179, "longitude": 7.4927, "latitude": 46.2538, "geo_altitude": 9867.9, "on_ground": false, "velocity": 228.36, "